# TALLER 2: Modelo de Espacio Vectorial
Objetivo 
Comprender y aplicar el Modelo de Espacio Vectorial para el ranking de documentos basado en la similitud de contenido con una consulta dada.

Parte 1: Preparacion de Datos

In [18]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from collections import Counter, defaultdict

# Textos sobre Madrid y Dortmund
textos = {
    'Texto1': "Historia de Madrid: Madrid, la capital de España, tiene una rica historia que se remonta al siglo IX. La ciudad fue fundada por los musulmanes, y su nombre proviene del árabe “Majrit” que significa “fuente de agua”. A lo largo de los siglos, Madrid se ha convertido en un centro político, económico y cultural de España.",
    'Texto2': "El Palacio Real: El Palacio Real de Madrid es una de las residencias oficiales de la familia real española, aunque ahora se usa principalmente para ceremonias de estado. Con más de 3,000 habitaciones, es el palacio real más grande de Europa Occidental y un testimonio del esplendor y la historia de España.",
    'Texto3': "Museo del Prado: El Museo del Prado es uno de los museos más famosos del mundo y un destino imprescindible para los amantes del arte que visitan Madrid. Alberga una impresionante colección de obras de Velázquez, Goya, El Greco y otros maestros europeos.",
    'Texto4': "Parque del Retiro: El Parque del Retiro es el pulmón verde de Madrid. Antiguamente perteneciente a la monarquía española, este parque de 118 hectáreas ofrece un respiro de la vida urbana con sus estanques, jardines y el Palacio de Cristal.",
    'Texto5': "Gastronomía Madrileña: Madrid es famosa por su vibrante escena gastronómica que incluye desde tapas en pequeños bares hasta restaurantes de alta cocina. Platos típicos como el cocido madrileño y las patatas bravas son esenciales para cualquier visitante.",
    'Texto6': "Gran Vía: La Gran Vía es una de las calles más famosas de Madrid, conocida por sus tiendas, teatros y arquitectura emblemática. Este bullicioso bulevar es un punto de encuentro para turistas y locales por igual.",
    'Texto7': "Real Madrid y el Estadio Santiago Bernabéu: El Real Madrid, uno de los clubes de fútbol más famosos y exitosos del mundo, juega sus partidos en el Estadio Santiago Bernab´eu. Este estadio es una meca para los aficionados al fútbol y ofrece tours que permiten a los visitantes explorar su rica historia.",
    'Texto8': "Navidad en Madrid: Durante la temporada navideña, Madrid se transforma con mercados festivos, luces deslumbrantes y decoraciones. La Plaza Mayor se convierte en un enorme mercado de Navidad, donde se pueden encontrar todo tipo de regalos y delicias tradicionales.",
    'Texto9': "Nochevieja en Puerta del Sol: La Puerta del Sol es el corazón de Madrid y el lugar donde miles se reúnen cada Nochevieja para dar la bienvenida al año nuevo. Comer las doce uvas de la suerte al son de las campanadas del reloj es una tradición que todos deberían experimentar.",
    'Texto10': "Madrid Río: Madrid Río es un parque lineal a lo largo del río Manzanares y un ejemplo perfecto de cómo la ciudad ha transformado áreas industriales en espacios verdes vibrantes. Es un lugar popular para caminar, andar en bicicleta y disfrutar de actividades al aire libre.",
    'Texto11': "Historia de Dortmund: Dortmund, situada en la región del Ruhr en Alemania, tiene una historia que se remonta al año 882. Originalmente una pequeña villa, Dortmund creció para convertirse en una importante ciudad industrial, especialmente conocida por su producción de acero y cerveza.",
    'Texto12': "Westfalenpark: Westfalenpark es uno de los parques urbanos más grandes de Europa y un lugar destacado para los visitantes en Dortmund. El parque es famoso por su Rosarium, que alberga una de las colecciones más extensas de rosas en el mundo, y la torre Florian, que ofrece vistas panorámicas de la ciudad.",
    'Texto13': "Museo del Fútbol Alemán: Inaugurado en 2015, el Museo del Fútbol Alemán ofrece una experiencia interactiva que celebra la rica historia del fútbol alemán. Ubicado en Dortmund, el museo atrae a aficionados de todo el mundo y destaca por sus exposiciones que cuentan desde los primeros días del deporte hasta los triunfos en Copas del Mundo.",
    'Texto14': "Mercado de Navidad de Dortmund: El Mercado de Navidad de Dortmund es uno de los más grandes y más visitados de Alemania. Con su gigantesco árbol de Navidad, posiblemente el más alto de los mercados navideños del país, y más de 300 puestos que venden artesanías, alimentos y bebidas tradicionales, es un evento imperdible durante la temporada festiva.",
    'Texto15': "U-Tower - Centro de Arte y Creatividad: El U-Tower, una antigua cervecería reconvertida, es ahora un centro dinámico de arte y creatividad. Alberga el Museo Ostwall, espacios de exposiciones y estudios de artistas, y es un símbolo de la transformación cultural de Dortmund.",
    'Texto16': "Borussia Dortmund y el Signal Iduna Park: Borussia Dortmund es uno de los clubes de fútbol más populares y exitosos de Alemania. El equipo juega sus partidos en casa en el Signal Iduna Park, conocido por su increíble atmósfera y el famoso “Muro Amarillo”, la tribuna sur que alberga a 25,000 aficionados apasionados.",
    'Texto17': "La cervecería Dortmund: Dortmund tiene una larga tradición cervecera, y la visita a una de sus muchas cervecerías es una parada obligatoria. Las cervecerías ofrecen tours que incluyen degustaciones de cervezas locales como Dortmunder Export, una lager que es un orgullo de la ciudad.",
    'Texto18': "La Reinoldikirche: La iglesia de San Reinold es el edificio más antiguo de Dortmund, datando del siglo XIII. Es un punto de referencia histórico importante y ofrece a los visitantes una mirada al pasado medieval de la ciudad.",
    'Texto19': "Dortmund durante la Segunda Guerra Mundial: Dortmund fue un objetivo significativo durante los bombardeos de la Segunda Guerra Mundial debido a su industria pesada. Hoy en día, varios monumentos y museos ofrecen reflexiones sobre este período y sus efectos en la ciudad.",
    'Texto20': "Zoológico de Dortmund: El Zoológico de Dortmund es un lugar excelente para familias y amantes de la naturaleza. Con más de 1,500 animales y un enfoque especial en la conservación de especies amenazadas, el zoológico proporciona una experiencia educativa y entretenida.",
}

# Categorias
cat = ["Turismo", "Historia", "Guerra", "Fútbol", "Realeza", "Gastronomía", "Arte y Cultura", "Naturaleza y Medio Ambiente", "Arquitectura", "Comercio", "Festividades", "Transporte", "Educación", "Tecnología", "Deportes (otros)", "Religión", "Ocio y Entretenimiento", "Salud y Bienestar", "Industria", "Urbanismo"]


# Generar palabras relacionadas para todas las categorías

categorias = {
    "Turismo": ["madrid", "visitan", "turistas", "turismo", "visita", "excursión", "aventura", "viaje", "destino", "tour operador", "guía turístico", "atractivo turístico", "alojamiento", "turismo rural", "turismo de aventura"],
    "Historia": ["historia", "fundada", "siglo", "histórico", "monarquía", "antigüedad", "prehistoria", "Edad Antigua", "Edad Media", "Edad Moderna", "Edad Contemporánea", "revolución", "independencia", "revolución industrial", "Revolución Francesa"],
    "Guerra": ["guerra", "batalla", "segunda guerra mundial", "bombardeos", "soldados", "conflicto armado", "batalla", "guerrero", "ejército", "estrategia militar", "armamento", "tregua", "violencia", "paz", "conquista"],
    "Fútbol": ["fútbol", "estadio", "real madrid", "borussia dortmund", "partidos", "jugador", "equipo", "entrenador", "árbitro", "estadio", "campeonato", "liga", "copa", "gol", "afición", "Madrid", "Dortmund"],
    "Realeza": ["realeza", "familia real", "palacio", "rey", "reina", "príncipe", "princesa", "monarquía", "corona", "trono", "dinastía", "palacio real", "protocolo"],
    "Gastronomía": ["gastronomía", "comida", "platos", "cocina", "restaurantes", "plato típico", "chef", "receta", "degustación", "restaurante", "comida callejera", "gastronomía internacional", "vinoteca", "cocina gourmet", "cata de vinos"],
    "Arte y Cultura": ["arte", "cultura", "museo", "exposición", "colección", "obra de arte", "artista", "movimiento artístico", "patrimonio cultural", "ballet", "teatro", "literatura", "cultura popular"],
    "Naturaleza y Medio Ambiente": ["parque", "naturaleza", "medio ambiente", "jardines", "animales","parque nacional", "reserva natural", "biodiversidad", "ecosistema", "flora", "fauna", "conservación ambiental", "reciclaje", "contaminación", "desarrollo sostenible"],
    "Arquitectura": ["arquitectura", "edificio", "torre", "iglesia", "catedral", "monumento", "rascacielos", "arco", "columna", "fachada", "arquitectura moderna", "arquitectura histórica", "diseño urbano", "patrimonio arquitectónico"],
    "Comercio": ["comercio", "mercado", "tiendas", "negocios", "productos", "comercio internacional", "negocio", "comerciante", "producto", "servicio", "economía", "exportación", "importación"],
    "Festividades": ["navidad", "festividades", "nochevieja", "celebración", "decoraciones", "festividad religiosa", "carnaval", "feria", "desfile", "ritual", "tradición", "fiesta popular", "evento cultural", "día festivo"],
    "Transporte": ["transporte", "caminar", "bicicleta", "viajar", "tren", "vehículo", "transporte público", "infraestructura de transporte", "aeropuerto", "estación de tren", "puerto", "tráfico", "conductor", "peatón", "movilidad urbana"],
    "Educación": ["educación", "escuela", "universidad", "estudios", "conocimiento", "universidad", "maestro", "alumno", "aprendizaje", "educación a distancia", "formación profesional", "aula", "examen", "programa educativo"],
    "Tecnología": ["tecnología", "innovación", "digital", "internet", "móvil", "innovación tecnológica", "dispositivo móvil", "red social", "aplicación móvil", "inteligencia artificial", "realidad virtual", "tecnología wearable", "software", "hardware", "ciberseguridad"],
    "Deportes (otros)": ["deportes", "atletismo", "baloncesto", "tenis", "competición", "natación", "ciclismo", "gimnasia", "boxeo", "rugby", "voleibol", "hockey sobre hielo"],
    "Religión": ["religión", "iglesia", "cristiana", "santos", "monjes","templo", "mezquita", "sinagoga", "oración", "creencia", "sacerdote", "festividad religiosa", "peregrinación", "fe"],
    "Ocio y Entretenimiento": ["ocio", "entretenimiento", "teatro", "cine", "juegos", "concierto", "parque de atracciones", "zoológico", "videojuego", "lectura", "pasatiempo", "karaoke", "baile"],
    "Salud y Bienestar": ["salud", "bienestar", "hospital", "medicina", "enfermedad","salud mental", "nutrición", "ejercicio físico", "medicina preventiva", "bienestar emocional", "terapia", "meditación", "vida saludable", "sueño reparador", "spa"],
    "Industria": ["industria", "fábrica", "producción", "manufactura", "trabajadores", "fábrica", "producción industrial", "sector industrial", "innovación tecnológica", "proceso productivo", "trabajador", "maquinaria", "cadena de producción", "industria automotriz"],
    "Urbanismo": ["urbanismo", "ciudad", "infraestructura", "diseño urbano", "planificación", "planificación urbana", "desarrollo urbano", "espacio público", "movilidad urbana", "arquitectura urbana", "sostenibilidad urbana", "diseño urbano", "zonificación urbana", "infraestructura urbana"]
}

# Consultas
consultas = [
    "Historia medieval de las ciudades europeas",
    "Principales destinos turísticos en Europa",
    "Influencia de la realeza en la cultura europea",
    "Importancia de los parques urbanos en las ciudades",
    "Gastronomía típica en capitales europeas",
    "Eventos deportivos icónicos en Europa",
    "Celebraciones de Navidad en ciudades europeas",
    "Museos de arte importantes en Europa",
    "Efectos de la Segunda Guerra Mundial en ciudades europeas",
    "Arquitectura histórica en ciudades europeas",
    "Clubes de fútbol famosos y sus estadios en Europa",
    "Transformación urbana y regeneración de espacios",
    "Tradición cervecera en ciudades europeas",
    "Mercados y comercio tradicional en Europa",
    "Centros de arte y creatividad en ciudades modernas",
    "Actividades de ocio y entretenimiento en ciudades metropolitanas",
    "Conservación de la naturaleza y la vida silvestre en zonas urbanas",
    "Planificación de eventos culturales y festivales en ciudades",
    "Desarrollo del transporte y la infraestructura urbana",
    "Impacto de la tecnología en la vida urbana"
]

La función `calcular_frecuencias` recibe dos parámetros: `textos` y `categorias`. Esta función se encarga de calcular las frecuencias de las categorías en los textos proporcionados.

Dentro de la función, se crea un diccionario llamado `frecuencias` utilizando `defaultdict(Counter)`. Esto permite tener un diccionario con valores predeterminados de tipo `Counter`, que es una subclase de `dict` especializada en contar elementos.

A continuación, se itera sobre cada texto en el diccionario `textos`. Para cada texto, se realiza una limpieza y tokenización del contenido utilizando expresiones regulares. Las palabras se convierten a minúsculas y se extraen utilizando la expresión regular `\b\w+\b`, que busca palabras completas.

Luego, se itera sobre cada categoría en el diccionario `categorias`. Para cada categoría, se itera sobre las palabras clave asociadas a esa categoría. Se cuenta la cantidad de veces que cada palabra clave aparece en el texto y se incrementa el contador correspondiente en el diccionario `frecuencias`.

Finalmente, se devuelve el diccionario `frecuencias` que contiene las frecuencias de las categorías en los textos.

In [19]:
# Función para calcular frecuencias de categorías en textos
def calcular_frecuencias(textos, categorias):
    frecuencias = defaultdict(Counter)
    for nombre_texto, contenido in textos.items():
        # Limpiar y tokenizar el texto
        palabras = re.findall(r'\b\w+\b', contenido.lower())
        for categoria, palabras_clave in categorias.items():
            for palabra in palabras_clave:
                frecuencias[nombre_texto][categoria] += palabras.count(palabra)
    return frecuencias

# Calcular frecuencias
frecuencias = calcular_frecuencias(textos, categorias)

In [20]:
frecuencias

defaultdict(collections.Counter,
            {'Texto1': Counter({'Historia': 4,
                      'Turismo': 3,
                      'Urbanismo': 1,
                      'Guerra': 0,
                      'Fútbol': 0,
                      'Realeza': 0,
                      'Gastronomía': 0,
                      'Arte y Cultura': 0,
                      'Naturaleza y Medio Ambiente': 0,
                      'Arquitectura': 0,
                      'Comercio': 0,
                      'Festividades': 0,
                      'Transporte': 0,
                      'Educación': 0,
                      'Tecnología': 0,
                      'Deportes (otros)': 0,
                      'Religión': 0,
                      'Ocio y Entretenimiento': 0,
                      'Salud y Bienestar': 0,
                      'Industria': 0}),
             'Texto2': Counter({'Realeza': 3,
                      'Turismo': 1,
                      'Historia': 1,
                      'Guerr

La función `construir_matriz` toma como entrada un diccionario de frecuencias y un diccionario de categorías. Esta función construye una matriz de términos-documento donde cada fila representa un texto y cada columna representa una categoría. Los valores de la matriz corresponden a las frecuencias de las categorías en los textos.

La función comienza obteniendo los nombres de los textos y las categorías a partir de los diccionarios de frecuencias y categorías. Luego, se crea una matriz de ceros con dimensiones determinadas por la cantidad de textos y categorías.

A continuación, se itera sobre los nombres de los textos y las categorías. Para cada combinación de texto y categoría, se asigna el valor de frecuencia correspondiente en la matriz.

Finalmente, la función devuelve la matriz de términos-documento, así como los nombres de los textos y las categorías.


In [21]:
# Construcción de la matriz de términos-documento
def construir_matriz(frecuencias, categorias):
    nombres_textos = list(frecuencias.keys())
    nombres_categorias = list(categorias.keys())
    matriz = np.zeros((len(nombres_textos), len(nombres_categorias)))

    for i, nombre_texto in enumerate(nombres_textos):
        for j, nombre_categoria in enumerate(nombres_categorias):
            matriz[i, j] = frecuencias[nombre_texto][nombre_categoria]

    return matriz, nombres_textos, nombres_categorias

matriz, nombres_textos, nombres_categorias = construir_matriz(frecuencias, categorias)

In [22]:
print("Matriz de términos-documento:")
print(matriz)

Matriz de términos-documento:
[[3. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 0. 0. 0. 0. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 2. 0. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 1. 0. 9. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 3. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 3. 0. 0. 0. 0.

La función `vectorizar_consulta` toma una consulta y un diccionario de categorías como entrada. Su objetivo es convertir la consulta en un vector numérico que representa la frecuencia de las palabras clave de cada categoría en la consulta.

La función comienza creando un vector de ceros con una longitud igual al número de categorías. Luego, se realiza una limpieza y tokenización de la consulta utilizando expresiones regulares para obtener las palabras individuales.

A continuación, se itera sobre cada categoría en el diccionario de categorías. Para cada categoría, se itera sobre las palabras clave asociadas a esa categoría. Se cuenta la cantidad de veces que cada palabra clave aparece en la consulta y se incrementa el valor correspondiente en el vector.

Finalmente, la función devuelve el vector resultante que representa la frecuencia de las palabras clave de cada categoría en la consulta.

In [23]:
# Vectorización de las consultas
def vectorizar_consulta(consulta, categorias):
    vector = np.zeros(len(categorias))
    palabras = re.findall(r'\b\w+\b', consulta.lower())
    for i, (categoria, palabras_clave) in enumerate(categorias.items()):
        for palabra in palabras_clave:
            vector[i] += palabras.count(palabra)
    return vector

La función `calcular_similitudes` calcula las similitudes coseno entre una matriz y un vector de consulta utilizando la función "cosine_similarity" de la biblioteca scikit-learn. Devuelve un arreglo de similitudes.

In [24]:
# Cálculo de similitudes coseno
def calcular_similitudes(matriz, vector_consulta):
    similitudes = cosine_similarity(matriz, vector_consulta.reshape(1, -1)).flatten()
    return similitudes

Se hace el llamado a las funciones `vectorizar_consulta` y `calcular_similitudes` para procesar cada consulta, se imprime el ranking de documentos correpondiente a cada consulta con su porcentaje de similitud.

In [25]:
# Procesar todas las consultas y mostrar resultados
for consulta in consultas:
    vector_consulta = vectorizar_consulta(consulta, categorias)
    similitudes = calcular_similitudes(matriz, vector_consulta)
    resultados = sorted(zip(nombres_textos, similitudes), key=lambda x: x[1], reverse=True)
    print(f"Consulta: {consulta}")
    for nombre_texto, similitud in resultados:
        print(f"  {nombre_texto}: {similitud:.4f}")
    print("\n")

Consulta: Historia medieval de las ciudades europeas
  Texto11: 0.8165
  Texto1: 0.7845
  Texto18: 0.6325
  Texto2: 0.3015
  Texto13: 0.2294
  Texto4: 0.2132
  Texto7: 0.1078
  Texto3: 0.0000
  Texto5: 0.0000
  Texto6: 0.0000
  Texto8: 0.0000
  Texto9: 0.0000
  Texto10: 0.0000
  Texto12: 0.0000
  Texto14: 0.0000
  Texto15: 0.0000
  Texto16: 0.0000
  Texto17: 0.0000
  Texto19: 0.0000
  Texto20: 0.0000


Consulta: Principales destinos turísticos en Europa
  Texto1: 0.0000
  Texto2: 0.0000
  Texto3: 0.0000
  Texto4: 0.0000
  Texto5: 0.0000
  Texto6: 0.0000
  Texto7: 0.0000
  Texto8: 0.0000
  Texto9: 0.0000
  Texto10: 0.0000
  Texto11: 0.0000
  Texto12: 0.0000
  Texto13: 0.0000
  Texto14: 0.0000
  Texto15: 0.0000
  Texto16: 0.0000
  Texto17: 0.0000
  Texto18: 0.0000
  Texto19: 0.0000
  Texto20: 0.0000


Consulta: Influencia de la realeza en la cultura europea
  Texto15: 0.6708
  Texto2: 0.6396
  Texto3: 0.5657
  Texto13: 0.4867
  Texto4: 0.3015
  Texto1: 0.0000
  Texto5: 0.0000
  Texto6: 0

# Presentación de Resultados
## Descripción de Resultados

En este trabajo, hemos procesado un conjunto de textos sobre Madrid y Dortmund y calculado las similitudes entre estos textos y una serie de consultas usando un modelo de espacio vectorial. Los resultados obtenidos indican qué tan relevante es cada texto para cada consulta, basado en las categorías y palabras clave que se han definido.


## Explicación del Ranking de Textos
Cada consulta es representada como un vector de categorías. Calculamos la similitud coseno entre este vector y los vectores de las categorías para cada texto, lo que nos permite ordenar los textos según su relevancia para la consulta.


A continuación, se presenta un ejemplo detallado de los resultados para una consulta específica:


### Ejemplo de Resultados para una Consulta

Consulta: "Historia medieval de las ciudades europeas"


Resultados:

Texto11: 0.8165

Texto1: 0.7845

Texto18: 0.6325

Texto2: 0.3015

Texto13: 0.2294

Texto4: 0.2132

Texto7: 0.1078

## Análisis del Ranking:

Texto11: Este texto trata sobre la historia de Dortmund, mencionando su origen en el año 882. Es el más relevante para la consulta debido a su contenido histórico, especialmente relacionado con el período medieval.

Texto1: Este texto trata sobre la historia de Madrid, mencionando su fundación en el siglo IX y su evolución histórica. Es muy relevante para la consulta debido a las palabras clave relacionadas con "historia" y "medieval".

Texto18: Este texto trata sobre la iglesia de San Reinold en Dortmund, que data del siglo XIII. La mención del siglo XIII y la referencia a un edificio histórico aumentan su relevancia.

Texto2: Describe el Palacio Real de Madrid y su historia. Aunque no es específicamente medieval, su contenido histórico general contribuye a su relevancia.

Texto13: Trata sobre el Museo del Fútbol Alemán en Dortmund. La conexión con la historia de la ciudad le da cierta relevancia, aunque no es medieval.

Texto4: Se centra en el Parque del Retiro en Madrid. Su relevancia es menor ya que no está directamente relacionado con la historia medieval.

Texto7: Habla sobre el Real Madrid y el Estadio Santiago Bernabéu. La relevancia es baja ya que el contenido está más enfocado en el deporte moderno que en la historia medieval.

## Discusión de Factores que Influyen en el Ranking
1. Peso de las Palabras:
El peso de las palabras clave en cada categoría juega un papel crucial en determinar la relevancia de un texto. Por ejemplo, en la consulta "Historia medieval de las ciudades europeas", las palabras como "historia", "siglo", "fundada", y "medieval" tienen un gran peso. Si una categoría incluye palabras con mayor frecuencia en un texto, ese texto se considerará más relevante.

2. Elección de Categorías:
La elección de las categorías también afecta significativamente los resultados. Las categorías deben ser representativas del contenido de los textos y las consultas. Por ejemplo, en esta implementación, las categorías como "Historia", "Realeza", y "Arquitectura" ayudan a capturar temas históricos y culturales de las ciudades. Si se eligieran categorías menos representativas, los resultados del ranking podrían no reflejar adecuadamente la relevancia del contenido.

3. Distribución de Frecuencias:
La distribución de las frecuencias de las palabras clave en los textos influye en la similitud calculada. Textos con una alta frecuencia de palabras clave relevantes obtendrán un mayor puntaje de similitud, mientras que textos con pocas palabras clave relevantes obtendrán un puntaje más bajo.

4. Longitud de los Textos:
La longitud de los textos puede influir en la cantidad de palabras clave presentes. Textos más largos tienen más oportunidades de contener palabras clave, lo que podría aumentar su relevancia, aunque esto también podría ser un efecto no deseado si el texto es redundante o no está enfocado.

5. Contexto de las Palabras:
El contexto en el que aparecen las palabras clave también es importante. Nuestro enfoque actual no considera la semántica, por lo que la relevancia se basa únicamente en la frecuencia de las palabras. Un análisis semántico más avanzado podría mejorar la precisión de la relevancia detectada.

## Conclusión
El modelo de espacio vectorial y el uso de la similitud coseno nos permiten medir y comparar la relevancia de diferentes textos frente a una serie de consultas predefinidas. La elección cuidadosa de categorías y palabras clave es crucial para obtener resultados precisos y relevantes.